In [ ]:
!pip install google-generativeai


In [ ]:
import base64
import google.generativeai as genai


In [ ]:
genai.configure(api_key="")

model = genai.GenerativeModel(
    model_name="models/gemini-3-flash-preview"
)


In [ ]:
def telugu_stt(audio_path):
    """
    Converts Telugu audio to Telugu text using Gemini
    """
    with open(audio_path, "rb") as f:
        audio_bytes = f.read()

    audio_base64 = base64.b64encode(audio_bytes).decode("utf-8")

    response = model.generate_content([
        {
            "mime_type": "audio/mp4",
            "data": audio_base64
        },
        "తెలుగు ఆడియోను ఖచ్చితంగా లిఖితం చేయండి."
    ])

    return response.text.strip()


In [ ]:
class FailureHandler:
    """
    Handles STT failure and missing information failure
    """

    def stt_failure(self, text):
        if not text or len(text) < 5:
            return {
                "status": "FAIL",
                "message": "❌ మీ మాట స్పష్టంగా వినిపించలేదు. దయచేసి మళ్లీ చెప్పండి."
            }
        return {"status": "OK"}

    def missing_info(self, memory):
        required_fields = ["age", "income", "state"]
        missing = [f for f in required_fields if f not in memory]

        if missing:
            questions = {
                "age": "➡️ మీ వయస్సు ఎంత?",
                "income": "➡️ మీ వార్షిక ఆదాయం ఎంత?",
                "state": "➡️ మీరు ఏ రాష్ట్రంలో నివసిస్తున్నారు?"
            }

            return {
                "status": "FAIL",
                "message": "పథకాన్ని గుర్తించడానికి కొంత సమాచారం అవసరం.",
                "followups": [questions[m] for m in missing]
            }

        return {"status": "OK"}


In [ ]:
def extract_memory(text, memory):
    """
    Extracts info from Telugu text (simple logic for demo)
    """

    if "సంవత్సర" in text or "ఏళ్లు" in text:
        memory["age"] = "CAPTURED"

    if "ఆదాయం" in text or "లక్ష" in text:
        memory["income"] = "CAPTURED"

    if "తెలంగాణ" in text or "ఆంధ్ర" in text:
        memory["state"] = "CAPTURED"

    return memory


In [ ]:
def govt_scheme_agent():
    return "✅ మీ సమాచారం ఆధారంగా తగిన ప్రభుత్వ పథకాలను సూచించవచ్చు."


In [50]:
audio_files = ["a1.mp4","a2.mp4","a3.mp4", "a4.mp4", "a5.mp4"]

failure = FailureHandler()
memory = {}

for audio in audio_files:
    print("\n===================================")
    print(f"🎧 Processing Audio: {audio}")

    # 1️⃣ STT
    text = telugu_stt(audio)
    print("📝 STT Output:", text)

    # 2️⃣ STT Failure Check
    check = failure.stt_failure(text)
    if check["status"] == "FAIL":
        print(check["message"])
        continue

    # 3️⃣ Save to Memory
    memory = extract_memory(text, memory)
    print("🧠 Memory:", memory)

    # 4️⃣ Missing Info Failure
    check = failure.missing_info(memory)
    if check["status"] == "FAIL":
        print("⚠️", check["message"])
        for q in check["followups"]:
            print(q)
        continue

    # 5️⃣ Tool Call (Success Path)
    response = govt_scheme_agent()
    print(response)



🎧 Processing Audio: a1.mp4


⏳ Quota hit. Retrying in 10s...


⏳ Quota hit. Retrying in 20s...
⚠️ Using fallback STT text
📝 STT Output: నాకు ప్రభుత్వ పథకం కావాలి
🧠 Memory: {}
⚠️ పథకాన్ని గుర్తించడానికి కొంత సమాచారం అవసరం.
➡️ మీ వయస్సు ఎంత?
➡️ మీ వార్షిక ఆదాయం ఎంత?
➡️ మీరు ఏ రాష్ట్రంలో నివసిస్తున్నారు?

🎧 Processing Audio: a2.mp4


⏳ Quota hit. Retrying in 10s...


⏳ Quota hit. Retrying in 20s...
⚠️ Using fallback STT text
📝 STT Output: 
❌ మీ మాట స్పష్టంగా వినిపించలేదు. దయచేసి మళ్లీ చెప్పండి.

🎧 Processing Audio: a3.mp4


...

⏳ Quota hit. Retrying in 20s...
⚠️ Using fallback STT text
📝 STT Output: నా వయస్సు 35 సంవత్సరాలు
🧠 Memory: {'age': 'CAPTURED'}
⚠️ పథకాన్ని గుర్తించడానికి కొంత సమాచారం అవసరం.
➡️ మీ వార్షిక ఆదాయం ఎంత?
➡️ మీరు ఏ రాష్ట్రంలో నివసిస్తున్నారు?

🎧 Processing Audio: a4.mp4


⏳ Quota hit. Retrying in 10s...


⏳ Quota hit. Retrying in 20s...
⚠️ Using fallback STT text
📝 STT Output: 
❌ మీ మాట స్పష్టంగా వినిపించలేదు. దయచేసి మళ్లీ చెప్పండి.

🎧 Processing Audio: a5.mp4


⏳ Quota hit. Retrying in 10s...


⏳ Quota hit. Retrying in 20s...
⚠️ Using fallback STT text
📝 STT Output: నా ఆదాయం రెండు లక్షలు మరియు నేను తెలంగాణలో ఉంటాను
🧠 Memory: {'age': 'CAPTURED', 'income': 'CAPTURED', 'state': 'CAPTURED'}
✅ మీ సమాచారం ఆధారంగా తగిన ప్రభుత్వ పథకాలను సూచించవచ్చు.
